In [18]:
import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec
import os

In [2]:
%pwd

'/Users/drjodyannjones/Documents/MLEngineering/End-to-End-Cancer-Classification-Project/research'

In [3]:
os.chdir("/Users/drjodyannjones/Documents/MLEngineering/End-to-End-Cancer-Classification-Project")

In [4]:
pwd

'/Users/drjodyannjones/Documents/MLEngineering/End-to-End-Cancer-Classification-Project'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/drjodyannjones/End-to-End-Cancer-Classification-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="drjodyannjones"
os.environ["MLFLOW_TRACKING_PASSWORD"]="f23438a79c7a32d04ef75d1a26b86e46f5765e22"

In [6]:
import tensorflow as tf

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from CNNClassifier.constants import*
from CNNClassifier.utils.common import read_yaml, create_directories, save_json

In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            create_directories([self.config.artifacts_root])
            
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/drjodyannjones/End-to-End-Cancer-Classification-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [27]:
import numpy as np
from mlflow.types import TensorSpec
import mlflow
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec
import tensorflow as tf
from urllib.parse import urlparse
from pathlib import Path

class Evaluation:
    def __init__(self, config):
        self.config = config
        
    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.30
        )
        
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    
    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()
        
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        # Assuming save_json is a function you've defined elsewhere
        save_json(path=Path("scores.json"), data=scores)
        
    def generate_signature(self):
        # Define the input schema with correct dimensions
        input_schema = Schema([
            TensorSpec(np.dtype(np.float32), [-1, 224, 224, 3]),
        ])
        
        # Define the output schema with correct dimensions
        output_schema = Schema([
            TensorSpec(np.dtype(np.float32), [-1, 2]),
        ])
        
        return ModelSignature(inputs=input_schema, outputs=output_schema)
        
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
            
            signature = self.generate_signature()
            
            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", signature=signature, registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model", signature=signature)


In [28]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
    
except Exception as e:
    raise e

[2024-03-30 17:49:45,248: INFO: common]: yaml file: config/config.yaml loaded successfully]
[2024-03-30 17:49:45,251: INFO: common]: yaml file: params.yaml loaded successfully]
[2024-03-30 17:49:45,252: INFO: common]: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 7s 966ms/step - loss: 7.6286 - accuracy: 0.5686
[2024-03-30 17:49:52,663: INFO: common]: json file saved at: scores.json]
[2024-03-30 17:49:52,715: WARNING: connectionpool]: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ConnectionResetError(54, 'Connection reset by peer')': /drjodyannjones/End-to-End-Cancer-Classification-Project.mlflow/api/2.0/mlflow/runs/create]
INFO:tensorflow:Assets written to: /var/folders/hf/r7nc2r613218f_z6flzjpd8h0000gn/T/tmp77sxl209/model/data/model/assets
[2024-03-30 17:49:54,725: INFO: builder_impl]: Assets written to: /var/folders/hf/r7nc2r613218f_z6flzjpd8h0000gn/T/tmp77sxl209/model/

Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/03/30 17:50:23 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 3
Created version '3' of model 'VGG16Model'.


In [24]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [25]:
input_shape = model.input_shape
print("Model input shape:", input_shape)

Model input shape: (None, 224, 224, 3)


In [26]:
output_shape = model.output_shape
print("Model output shape:", output_shape)

Model output shape: (None, 2)
